In [1]:
import math
import time
from serial import Serial
from IPython.display import clear_output
#import os
import cv2
ser = Serial('/dev/tty.usbmodem3301', 115200)
# Read Data from Other notebook RUN this first!
%store -r mdata
%store -r fducials
#mdata 
fducials

{'fd1': {'x': 1.0668, 'y': 1.4224}, 'fd2': {'x': 28.4988, 'y': 18.4404}}

In [2]:
# thx @flo ;-)

# my_fd0, my_fd1 obj with member x,y to fiducial positionen on pcb in mm from eagle or kicad
# r_fd0_x,r_fd0_y ,r_fd1_x, r_fd1_y. Fiducial Cam Position after calibration
# my_id0  obj with member x,y with native position in mm as input position
# return: cam position for driver

# native Stepposition * 1000
mpos_x = 300000
mpos_y = 300000

# some default sample fiducial native positions
fd1_pos_x = 159240
fd1_pos_y = 291160
fd2_pos_x = 318360
fd2_pos_y = 389420

pad_idx=0
is_driving = False

def driveToNative(x,y):
    global is_driving
    is_driving = True
    clear_output(wait=True)
    gcode = "1 " + str((int(x) - 300000) * -1 ) + ' ' + str(int(y - 300000)) + "\n"
    print("driveToNative",x,y,gcode)
    ser.write(gcode.encode('raw_unicode_escape'))
def readPosFromDriver():
    global mpos_x,mpos_y,is_driving
    if ser.in_waiting > 0:
        mpos = ser.read(ser.in_waiting).split()
        if mpos[0] == b'OK':
            mpos_x = int(mpos[2]) * -1 + 300000
            mpos_y = int(mpos[3]) + 300000
            print("mpos:",mpos_x,mpos_y)
        else:
            print("Error",mpos)
        is_driving = False
    
def driveTo(destPos):
    global fducials,fd1_pos_x,fd1_pos_y,fd2_pos_x,fd2_pos_y
    p = convertRect(fducials['fd1'],fducials['fd2'],destPos, fd1_pos_x,fd1_pos_y,fd2_pos_x,fd2_pos_y)
    driveToNative(p[0],p[1])

def convertRect(my_fd0,my_fd1,my_id0, 
                r_fd0_x = 159240,  r_fd0_y = 291160,
                r_fd1_x = 318360,  r_fd1_y = 389420):
    abDistance1 = math.sqrt( (my_fd1['x'] - my_fd0['x']) * (my_fd1['x'] - my_fd0['x']) + 
                             (my_fd1['y'] - my_fd0['y']) * (my_fd1['y'] - my_fd0['y']) )
    middle1_x = my_fd0['x'] + ((my_fd1['x'] - my_fd0['x']) / 2.0)
    middle1_y = my_fd0['y'] + ((my_fd1['y'] - my_fd0['y']) / 2.0)
    r1Local_a_x = my_fd0['x'] - middle1_x
    r1Local_a_y = my_fd0['y'] - middle1_y
    r1Local_b_x = my_fd1['x'] - middle1_x
    r1Local_b_y = my_fd1['y'] - middle1_y
    r1Local_p_x = my_id0['x'] - middle1_x
    r1Local_p_y = my_id0['y'] - middle1_y
    angle1 = math.atan2(r1Local_p_y, r1Local_p_x) - math.atan2(r1Local_a_y, r1Local_a_x)
    pLength1 = math.sqrt((r1Local_p_x * r1Local_p_x) + (r1Local_p_y * r1Local_p_y))
    angle_grad =  angle1 * (180.0 / math.pi)
    abDistance2 = math.sqrt( (r_fd1_x - r_fd0_x) * (r_fd1_x - r_fd0_x) + (r_fd1_y - r_fd0_y) * (r_fd1_y - r_fd0_y))
    middle2_x = r_fd0_x + ((r_fd1_x - r_fd0_x) / 2.0)
    middle2_y = r_fd0_y + ((r_fd1_y - r_fd0_y) / 2.0)
    r2Local_a_x = r_fd0_x - middle2_x
    r2Local_a_y = r_fd0_y - middle2_y
    r2Local_b_x = r_fd1_x - middle2_x
    r2Local_b_y = r_fd1_y - middle2_y
    p2Direction_x = r2Local_a_x * math.cos(angle1) - r2Local_a_y * math.sin(angle1)
    p2Direction_y = r2Local_a_x * math.sin(angle1) + r2Local_a_y * math.cos(angle1)
    pLength2 = math.sqrt((p2Direction_x * p2Direction_x) + (p2Direction_y * p2Direction_y))
    p2Direction_x = p2Direction_x / pLength2
    p2Direction_y = p2Direction_y / pLength2
    scaleFactor = abDistance2 / abDistance1
    p2_x = p2Direction_x * pLength1 * scaleFactor
    p2_y = p2Direction_y * pLength1 * scaleFactor
    p2_x = middle2_x + p2_x
    p2_y = middle2_y + p2_y
    return (p2_x,p2_y,angle_grad)


In [3]:
def drawText(img,x,y,text,color = (100,100,255), size = 1):
    p1 = ( int(x), int( y) )
    cv2.putText(img, text, p1 , cv2.FONT_HERSHEY_SIMPLEX, size, color, 2, cv2.LINE_AA)
def draw_crosshair(frame): #fadenkreuz, absehen
    h, w, ch = frame.shape
    crosshair_color = (0, 255, 255)
    cv2.line(img=frame, pt1=(0, int(h/2)), pt2=(w, int(h/2)), color=crosshair_color , thickness = 1, lineType = 8, shift = 0)
    cv2.line(img=frame, pt1=(int(w/2), 0), pt2=(int(w/2), h), color=crosshair_color , thickness = 1, lineType = 8, shift = 0)
    lines = 32
    for x in range(lines):
        cv2.line(img=frame, pt1=(int(w/lines*x), int(h/2-10) ), pt2=(int(w/lines*x), int(h/2+10) ), color=crosshair_color , thickness = 1, lineType = 8, shift = 0)
        cv2.line(img=frame, pt1=(int(w/2-10), int(h/lines*x) ), pt2=( int(w/2+10), int(h/lines*x) ), color=crosshair_color , thickness = 1, lineType = 8, shift = 0)
        cv2.circle(img=frame, center=(int(w/2), int(h/2) ), radius=100, color=crosshair_color, thickness = 1)
    drawText(frame,10,50,"# " + str(pad_idx) + " " + str(mpos_x) + " " + str(mpos_y))
    return frame

In [ ]:



loop_cnt=0
loop_wait=8

def onMouse(event, x, y, flags, param):
    global mpos_x,mpos_y
    if event == cv2.EVENT_LBUTTONDOWN:
        print('onMouse x = %d, y = %d'%(x, y))
        x_new_pos = mpos_x +  (x-320)*169 
        y_new_pos = mpos_y -  (y-240)*169
        driveToNative(x_new_pos ,y_new_pos)

cv2.namedWindow("preview")
cv2.setMouseCallback('preview', onMouse)
vc = cv2.VideoCapture(1)
rval, frame = vc.read()
autodrive = False

while True:
    readPosFromDriver()
    if frame is not None:   
        cv2.imshow("preview", frame)
        rval, frame = vc.read()
    frame = draw_crosshair(frame)
    key_input = cv2.waitKey(1) & 0xFF 
    if key_input == ord('q'): # quit
        break
    if key_input == ord('s'): # slow
        loop_wait = 8
    if key_input == ord('f'): # fast
        loop_wait = 0
    if key_input == ord('a'): # autodrive
        autodrive = True
    if key_input == ord('h'): # update fiducial 1 to current position
        fd1_pos_x = mpos_x
        fd1_pos_y = mpos_y
    if key_input == ord('j'): # update fiducial 2 to current position
        fd2_pos_x = mpos_x
        fd2_pos_y = mpos_y
    if key_input == ord('1'): # drive to fiducial 1
        driveToNative(fd1_pos_x,fd1_pos_y)
    if key_input == ord('2'): # drive to fiducial 1
        driveToNative(fd2_pos_x,fd2_pos_y)
    if key_input == ord('0'): # jump to 0,0 , reset pad index=0
        pad_idx = 0
        autodrive = False
        #os.system("printf '\a'") # or '\7'
        driveToNative(300000,300000)
        #ser.write("0 0 0\n".encode('raw_unicode_escape'))
    if key_input == ord('n'): # jump next pad
        autodrive = False
        driveTo(mdata[pad_idx])
        pad_idx=min(pad_idx+1,len(mdata)-1)
    if key_input == ord('p'): # jump prev pad
        autodrive = False
        driveTo(mdata[pad_idx])
        pad_idx=max(pad_idx-1,0)
    loop_cnt = loop_cnt+1
    if autodrive and pad_idx < (len(mdata)-1):
        if not is_driving and loop_cnt > loop_wait:
            pad_idx=min(pad_idx+1,len(mdata)-1)
            driveTo(mdata[pad_idx])
            loop_cnt=0
                

driveToNative 300000 300000 1 0 0

mpos: 300000 300000
